In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 11.1 그레디언트 소실과 폭주 문제

### 11.1.1 글로럿(xavier) 과 He 초기화

In [ ]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

In [ ]:
keras.layers.Dense(10,activation='relu', kernel_initializer=keras.initializers.he_normal)

fan-in 대신 fan-out 기반의 균등분포 he 초기화를 사용하고 싶을때

In [ ]:
init = keras.initializers.VarianceScaling(scale=2.,mode = 'fan_avg',
                                          distribution='uniform') 
# 잘 이해 안됨

In [ ]:
keras.layers.Dense(10,activation='relu',kernel_initializer=init)

### 11.1.2 수렴하지 않는 활성화 함수

다양한 Relu 계열의 함수들은 하이퍼파라미터 알파에 의에 나뉘어짐

LeakyReLu

In [ ]:
def leaky_relu(z,alpha=0.01):
  return np.maximum(z*alpha,z)

In [ ]:
z= np.linspace(-5,5,200)

In [ ]:
plt.plot(z,leaky_relu(z,0.05), "b-" , linewidth=2)
plt.plot([-5,5],[0,0],'k-')
plt.plot([0,0],[-0.5,4.2],'k-')
plt.grid(True)
props= dict(facecolor='black', shrink=0.1)
plt.annotate('Leak', xytext=(-3.5,0.5),xy=(-5,-0.2),arrowprops=props,fontsize=14, ha="center")
plt.title("Leaky ReLU activation function", fontsize=14)
plt.axis([-5, 5, -0.5, 4.2])

save_fig("leaky_relu_plot")
plt.show()

In [ ]:
[m for m in dir(keras.activations) if not m.startswith("_")]

In [ ]:
[m for m in dir(keras.layers) if "relu" in m.lower()]

In [ ]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full/255.
X_test = X_test/255.

X_valid,X_train = X_train_full[:5000], X_train_full[5000:]
y_valid,y_train = y_train_full[:5000], y_train_full[5000:]

LeakyReLu 를 활성화 함수로 사용한 신경망

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28,28]),
                                 keras.layers.Dense(300,kernel_initializer=keras.initializers.he_normal),
                                 keras.layers.LeakyReLU(), #리키렐루, 등은 층 뒤에 따로 추가해줘야함. 하이퍼파라미터로 LeakyReLu(alpha=0.2) 가능
                                 keras.layers.Dense(100,kernel_initializer=keras.initializers.he_normal),
                                 keras.layers.LeakyReLU(),
                                 keras.layers.Dense(10,activation='softmax')
])

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer= keras.optimizers.SGD(lr=1e-3),
              metrics = ['accuracy'])

In [ ]:
history= model.fit(X_train,y_train,epochs=10,
                   validation_data = (X_valid,y_valid))

PReLu를 사용한 신경망 (alpha를 역전파로 학습)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28,28]),
                                 keras.layers.Dense(300,kernel_initializer=keras.initializers.he_normal),
                                 keras.layers.PReLU(), 
                                 keras.layers.Dense(100,kernel_initializer=keras.initializers.he_normal),
                                 keras.layers.PReLU(),
                                 keras.layers.Dense(10,activation='softmax')
])

In [ ]:
model.compile(loss= keras.losses.sparse_categorical_crossentropy, optimizer= keras.optimizers.SGD(lr=1e-3),
              metrics=  ["accuracy"])

In [ ]:
history = model.fit(X_train,y_train,epochs=10,
                    validation_data = (X_valid,y_valid))

ELU

In [ ]:
def elu(z,alpha=1):
  return np.where(z<0, alpha*(np.exp(z)-1), z )

In [ ]:
plt.plot(z,elu(z),linewidth=2)
plt.plot([-5,5],[0,0],'k-')
plt.plot([-5,5],[-1,-1],'k--')
plt.plot([0,0],[-2.2,3.2],'k--')
plt.grid(True)
plt.title(r"ELU activation function ($\alpha=1$)", fontsize=14)
plt.axis([-5,5,-2.2,3.2])

save_fig("elu_plot")
plt.show()

SELU

In [ ]:
from scipy.special import erfc

# alpha와 scale은 평균 0과 표준 편차 1로 자기 정규화합니다
# (논문에 있는 식 14 참조):
alpha_0_1 = -np.sqrt(2 / np.pi) / (erfc(1/np.sqrt(2)) * np.exp(1/2) - 1)
scale_0_1 = (1 - erfc(1 / np.sqrt(2)) * np.sqrt(np.e)) * np.sqrt(2 * np.pi) * (2 * erfc(np.sqrt(2))*np.e**2 + np.pi*erfc(1/np.sqrt(2))**2*np.e - 2*(2+np.pi)*erfc(1/np.sqrt(2))*np.sqrt(np.e)+np.pi+2)**(-1/2)

In [ ]:
def selu(z, scale=scale_0_1, alpha=alpha_0_1):
    return scale * elu(z, alpha)

In [ ]:
plt.plot(z, selu(z), "b-", linewidth=2)
plt.plot([-5, 5], [0, 0], 'k-')
plt.plot([-5, 5], [-1.758, -1.758], 'k--')
plt.plot([0, 0], [-2.2, 3.2], 'k-')
plt.grid(True)
plt.title("SELU activation function", fontsize=14)
plt.axis([-5, 5, -2.2, 3.2])

save_fig("selu_plot")
plt.show()

표준화된 입력이 있다면 

selu는 각 층의 출력을 평균0 표준편차1로 유지시킴

In [ ]:
np.random.seed(42)
Z = np.random.normal(size=(500, 100)) # 표준화된 입력
for layer in range(1000):
    W = np.random.normal(size=(100, 100), scale=np.sqrt(1 / 100)) # selu는 LeCun 정규분포 초기화(fan-in)
    Z = selu(np.dot(Z, W))
    means = np.mean(Z, axis=0).mean()
    stds = np.std(Z, axis=0).mean()
    if layer % 100 == 0:
        print("Layer {}: mean {:.2f}, std deviation {:.2f}".format(layer, means, stds))

In [ ]:
layer = keras.layers.Dense(10,activation="selu",kernel_initializer=keras.initializers.lecun_normal) #selu는 lecun 정규분포 초기화

100개의 은닉층과 selu 활성화 함수를 사용한 fashion mnist

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation='selu', kernel_initializer=keras.initializers.lecun_normal))

for layer in range(99):
  model.add(keras.layers.Dense(100,activation='selu',kernel_initializer=keras.initializers.lecun_normal))
model.add(keras.layers.Dense(10,activation="softmax"))

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.SGD(1e-3),
              metrics = ['accuracy'])

 SELU는 모델 훈련시 입력특성이 반드시 표준화 돼있어야한다.

In [ ]:
pixel_means = X_train.mean(axis=0,keepdims=True)
pixel_stds = X_train.std(axis=0,keepdims=True)
X_train_scaled = (X_train-pixel_means)/pixel_stds
X_valid_scaled = (X_valid-pixel_means)/pixel_stds
X_test_scaled = (X_test- pixel_means)/pixel_stds

In [ ]:
history = model.fit(X_train_scaled,y_train,epochs=5,
                    validation_data=(X_valid_scaled,y_valid))

같은 Relu 구조와 비교 (selu vs relu)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation='relu', kernel_initializer=keras.initializers.he_normal))

for layer in range(99):
  model.add(keras.layers.Dense(100,activation='relu',kernel_initializer=keras.initializers.he_normal))
model.add(keras.layers.Dense(10,activation='softmax'))

In [ ]:
model.compile(loss= keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.SGD(lr=1e-3),
              metrics = ["accuracy"])

In [ ]:
history = model.fit(X_train_scaled, y_train,epochs=5,
                    validation_data = (X_valid_scaled,y_valid))
#relu 결과 val_accuracy: 0.7444... 그레디언트 손실, 폭주 발생

### 11.1.3 배치 정규화



**미니배치 입력의 스케일을 조정하고 이동시킴.**

역시나 그레디언트 소실, 폭발 문제를 해결하기 위함.

Q.테스트는 샘플 하나에 대한예측을 하는데 학습된 모델로부터어떻게 평균과 표준편차를 적용할 것인가?

1.훈련이 끝난후 전체 훈련셋을 통과시켜 대신사용

2.층의 입력평균과 표준편차의 이동평균을 훈련하는동안 추정함

In [ ]:
model =keras.models.Sequential([
                                keras.layers.Flatten(input_shape=[28,28]),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(300,activation='relu'),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(100,activation='relu'),
                                keras.layers.BatchNormalization(),
                                keras.layers.Dense(10,activation='softmax')

])

BN는 4개의 파라미터가 있는것을 확인가능.


ex)첫 정규화층 : 4x784 = 3136

In [ ]:
model.summary()

In [ ]:
bn1 =model.layers[1]
#두번째 레이어 불러움
[(var.name,var.trainable) for var in bn1.variables]
#2개의 학습가능한 스케일,출력이동 파라미터,
#2개의 학습 불가능한 파라미터 (추정을 위한 이동평균)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

활성화함수 전에 BN사용해보기.

사용전, 후 무엇이 좋은지는 직접 해보고 비교 추천.

In [ ]:
model= keras.models.Sequential([
                                
keras.layers.Flatten(input_shape=[28,28]),
keras.layers.BatchNormalization(),
keras.layers.Dense(300,use_bias=False),
#활성화함수 전에 사용
keras.layers.BatchNormalization(),
keras.layers.Activation('relu'),
keras.layers.Dense(100,use_bias=False),
#활성화함수 전에 사용
keras.layers.BatchNormalization(),
keras.layers.Activation('relu'),
keras.layers.Dense(10,activation='softmax')
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

### 11.1.4 그레디언트 클리핑

-1~1을 넘을 시 그레디언트를 잘라냄. 벡터의 방향을 바꿀수 있음.

In [ ]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)

-1~1을 넘을시 그레디언트 합을 1로 정규화함

In [ ]:
optimizer=  keras.optimizers.SGD(clipnorm=1.0)

# 11.2 사전훈련된 층 재사용하기

Fashion MNIST에서 샌달과 셔츠(5,6)를 제외한 8개 데이터셋으로 학습한 분류 모델을 이용해 샌달과 셔츠(5,6) 이진분류

패션 MNIST 훈련 세트를 두 개로 나누어 보죠:

X_train_A: 샌달과 셔츠(클래스 5와 6)을 제외한 모든 이미지

X_train_B: 샌달과 셔츠 이미지 중 처음 200개만 가진 작은 훈련 세트

In [ ]:
def split_dataset(X,y):
  y_5_or_6 = (y==5)|(y==6)
  y_A = y[~y_5_or_6]
  y_A[y_A>6] -=2 # 5,6 없어져서 당김.
  y_B = (y[y_5_or_6]==6).astype(np.float32) #5,6을 이진데이터로.
  return (X[~y_5_or_6], y_A),(X[y_5_or_6],y_B)

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

#B셋은 200개만 이용
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [ ]:
X_train_A.shape

In [ ]:
X_train_B.shape

In [ ]:
y_train_A[:30]

In [ ]:
y_train_B[:30]

모델A학습

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
model_A  = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
  model_A.add(keras.layers.Dense(n_hidden, activation='selu'))
model_A.add(keras.layers.Dense(8,activation='softmax'))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_A.fit(X_train_A,y_train_A,epochs=20,
                      validation_data = (X_valid_A,y_valid_A))

In [ ]:
model_A.save("my_model_A.h5")

모델 B학습

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

전이학습

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

B를 학습하면 A도학습되니깐 클론해둠.

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
#weights는 따로 설정해줘야함. 복사안됨.

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

In [ ]:
model_B.evaluate(X_test_B, y_test_B)

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

In [ ]:
(100 - 97.05) / (100 - 99.35)

# 11.3 고속 옵티마이저

### 11.3.1 모멘텀 최적화

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)                                                                 

### 11.3.2 네스테로프 가속 경사

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov = True)

11.3.3 AdaGrad

In [ ]:
optimizer = keras.optimizers.Adagrad(lr=0.001)

11.3.4 RMSProp

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9) #rho = 감쇠율

### 11.3.5 Adam , AdaMax, Nadam

Adam

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.001, beta_1 = 0.9, beta_2 = 0.999) # 1. 모멘텀감쇠 2. 스케일감쇠

AdaMax

In [ ]:
optimizer = keras.optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2 = 0.999)

Nadam

In [ ]:
optimizer = keras.optimizers.Nadam(lr=0.001, beta_1 = 0.9, beta_2 = 0.999)

### 11.3.6 학습률 스케줄링

거듭제곱 스케줄링


lr = lr0 / (1 + steps / s)**c

케라스는 c=1과 s = 1 / decay을 사용

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.01, decay = 1e-4)

지수 기반 스케줄링

lr = lr0 * (0.1) **(t/s)

In [ ]:
def exponential_decay_fn(epoch, lr):
    return lr * 0.1**(1 / 20)

콜백으로 만들기

In [ ]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
model = keras.models.Sequential([
                                keras.layers.Flatten(input_shape=[28,28]),
                                keras.layers.Dense(300,activation='selu',kernel_initializer=keras.initializers.lecun_normal),
                                keras.layers.Dense(100,activation='selu',kernel_initializer=keras.initializers.lecun_normal),
                                keras.layers.Dense(10,activation='softmax')
])

model.compile(loss=keras.losses.sparse_categorical_crossentropy, 
              optimizer = keras.optimizers.Nadam(), 
              metrics = ["accuracy"])

In [ ]:
history = model.fit(X_train,y_train,epochs=25,
                    validation_data = (X_valid, y_valid),
                    callbacks = [lr_scheduler])

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.epoch, history.history["lr"], "o-")
plt.axis([0,25-1,0,max(history.history["lr"])])
plt.xlabel("Epoch")
plt.ylabel("learning rate")
plt.grid(True)
plt.title("exponential scheduling")
plt.show()

구간별 고정 스케줄링

In [ ]:
def piecewise_constant_fn(epoch):
  if epoch<5:
    return 0.01
  elif epoch>15:
    return 0.005
  else:
    return 0.001

성능 기반 스케줄링

In [ ]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience =5)
#5번의 연속적인 에폭동안 val loss가 향상되지 않으면 lr = lr * 0.5

keras 스케줄러 

keras.optimzers.schedules에 있는 스케줄에서 하나를 고르고 생성한 후 옵티마이저에 추가

에폭이 아닌 스텝마다 decay 가능함.

keras 스케줄러를 이용한 지수 기반 스케줄링

In [ ]:
s = 20 * len(X_train) // 32 #20에폭 동안의 스텝 수

In [ ]:
learning_rate= keras.optimizers.schedules.ExponentialDecay(0.01, s , 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

keras 스케줄러를 이용한 구간별 스케줄링

In [ ]:
batch_size = 32
n_steps_per_epoch = len(X_train) // batch_size
#총 배치 수 나옴.
learning_rate = keras.optimizers.schedules.PiecewiseConstantDecay(
    
  boundaries = [5. * n_steps_per_epoch, 15. * n_steps_per_epoch],
  values =[0.01,0.005,0.001]

)

# 11.4 규제를 사용해 과대적합 피하기

$\ell_1$과 $\ell_2$ 규제

In [ ]:
layer = keras.layers.Dense(100,activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
#l1만 쓰고싶으면 kernel_regularizer=keras.regularizers.l1(0.1))
#l1,l2 함께 쓰고싶으면 kernel_regularizer=keras.regularizers.l1_l2(0.1,0.01))

### 11.4.2 드롭아웃

In [ ]:
model = keras.models.Sequential([
                                 
keras.layers.Flatten(input_shape = [28,28]),
keras.layers.Dropout(rate = 0.2),#drop-out layer
keras.layers.Dense(300,activation='elu',kernel_initializer=keras.initializers.he_normal),
keras.layers.Dropout(rate=0.2),
keras.layers.Dense(100,activation='elu',kernel_initializer=keras.initializers.he_normal),
keras.layers.Dropout(rate=0.2),
keras.layers.Dense(10, activation = "softmax")
])

model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=  keras.optimizers.Nadam(),
              metrics = ["accuracy"])
n_epochs=2

history = model.fit(X_train,y_train,epochs = n_epochs,
                    validation_data = (X_valid,y_valid)
                    )

알파 드롭아웃 (selu와 함께 사용하여 입력의 평균과 표준편차를 유지해줌)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.Flatten(input_shape=[28,28]),
                                 #알파 드롭아웃
                                 keras.layers.AlphaDropout(rate= 0.2),
                                 #selu, lecun와 함께 사용
                                 keras.layers.Dense(300,activation='selu', kernel_initializer=keras.initializers.lecun_normal),
                                keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
                                keras.layers.AlphaDropout(rate=0.2),
                                keras.layers.Dense(10, activation="softmax")
                                ])

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

드롭아웃 적용 시 train loss는 끝나고 비교해보는게 좋음

In [ ]:
model.evaluate(X_test_scaled,y_test)

In [ ]:
model.evaluate(X_train_scaled,y_train)

### 11.4.3 몬테 카를로 드롭아웃

일반적인 드롭아웃은 훈련시에만 드롭아웃을 사용. 테스트 시에는 하나의 모델로 드롭아웃 없이 예측. 따라서 동일한 입력에 대한 출력은 항상 같음.

몬테카를로 드롭아웃은 훈련과 예측 모두에서 드롭아웃을 사용하여 n번 예측 수행. 따라서 예측에 대한 확률값을 구할 수 있음.

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
y_probas =np.stack([model(X_test_scaled, training = True)
for sample in range(100)])
#model(X_test_scaled)는 model.predict와 거의비슷. 전자는 텐서를 반환. 
#후자는 넘파이 배열을 반환
#training=True로 하여 드롭아웃을 활성화함 --> 100개의 예측이 달라짐

y_proba = y_probas.mean(axis=0)
y_std = y_probas.std(axis=0)

In [ ]:
y_probas.shape
#100개의 모델. 10000개의 데이터, 10개의 클래스에 대한 예측값.

In [ ]:
np.round(model.predict(X_test_scaled[:1]), 2)
#모델 하나의 첫번째 샘플에 대한 클래스 확률 예측확인 (2번째에서 반올림)

In [ ]:
np.round(y_probas[:,:1], 2)
#100개 모델에서 첫번째 모델 예측

In [ ]:
np.round(y_proba[:1],2)

In [ ]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1],2)

In [ ]:
y_pred = np.argmax(y_proba, axis=1)

In [ ]:
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

11.4.4 맥스 노름

매 훈련 스텝이 끝난 후 w의 $\ell2$ 노름을 계산하고 r보다 클 시 w의 스케일을 조정

In [ ]:
layer = keras.layers.Dense(100,activation="selu",kernel_initializer="lecun_normal",
                           kernel_constraint = keras.constraints.max_norm(1.))

In [ ]:
from functools import partial

In [ ]:
MaxnormDense = partial(keras.layers.Dense,
                       activation="selu", kernel_initializer="lecun_normal",
                       kernel_constraint=keras.constraints.max_norm(1.))


In [ ]:
model = keras.models.Sequential([
                                 
          keras.layers.Flatten(input_shape=[28,28]),
          MaxnormDense(300),
          MaxnormDense(100),
          keras.layers.Dense(10,activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

# 연습문제 8

a.
문제: 100개의 뉴런을 가진 은닉층 20개로 심층 신경망을 만들어보세요(너무 많은 것 같지만 이 연습문제의 핵심입니다). He 초기화와 ELU 활성화 함수를 사용하세요.

b.
문제: Nadam 옵티마이저와 조기 종료를 사용하여 CIFAR10 데이터셋에 이 네트워크를 훈련하세요. keras.datasets.cifar10.load_ data()를 사용하여 데이터를 적재할 수 있습니다. 이 데이터셋은 10개의 클래스와 32×32 크기의 컬러 이미지 60,000개로 구성됩니다(50,000개는 훈련, 10,000개는 테스트). 따라서 10개의 뉴런과 소프트맥스 활성화 함수를 사용하는 출력층이 필요합니다. 모델 구조와 하이퍼파라미터를 바꿀 때마다 적절한 학습률을 찾아야 한다는 것을 기억하세요.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
(X_train_full,y_train_full),(X_test_full,y_test_full) = keras.datasets.cifar10.load_data()

In [ ]:
X_train_full.shape

In [ ]:
X_test_full.shape

In [ ]:
X_train_full.dtype

In [ ]:
X_train ,X_valid = X_train_full[5000:], X_train_full[:5000]
y_train,y_valid = y_train_full[5000:], y_train_full[:5000]

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))

for layer in range(20):
  model.add(keras.layers.Dense(100,
                               activation = keras.activations.elu,
                               kernel_initializer=keras.initializers.he_normal()))
  
model.add(keras.layers.Dense(10,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer= keras.optimizers.Nadam(5e-5),
              metrics =["accuracy"])

콜백들 만들기

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # 로그 저장을 위한 카운터
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [ ]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),callbacks=callbacks)

In [ ]:
%tensorboard --logdir=./my_cifar10_logs --port=6006

### C.문제: 배치 정규화를 추가하고 학습 곡선을 비교해보세요. 이전보다 빠르게 수렴하나요? 더 좋은 모델이 만들어지나요? 훈련 속도에는 어떤 영향을 미치나요?

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
smodel = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
#활성화 함수 전에 BN 추가
for layer in range(20):
  model.add(keras.layers.Dense(100,kernel_initializer = "he_normal"))
  model.add( keras.layers.BatchNormalization())         
  model.add(keras.layers.Activation(keras.activations.elu))       
model.add(keras.layers.Dense(10,activation=keras.activations.softmax))

In [ ]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(lr=5e-4),
              metrics = ["accuracy"])

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)


run_index = 1
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [ ]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

### d.
문제: 배치 정규화를 SELU로 바꾸어보세요. 네트워크가 자기 정규화하기 위해 필요한 변경 사항을 적용해보세요(즉, 입력 특성 표준화, 르쿤 정규분포 초기화, 완전 연결 층만 순차적으로 쌓은 심층 신경망 등).

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

#model
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for layer in range(20):
  model.add(keras.layers.Dense(100,
                               kernel_initializer='lecun_normal',
                               activation='selu')
                                )
model.add(keras.layers.Dense(10,activation='softmax'))

model.compile(loss= keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.Nadam(lr= 7e-4),
              metrics = ["accuracy"])

#callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # 모델을 훈련할 때마다 증가시킴
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]


X_means = X_train.mean(axis = 0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train-X_means) / X_stds
X_valid_scaled = (X_valid -X_means) / X_stds
X_test_scaled = (X_test_full - X_means) / X_stds

model.fit(X_train_scaled,y_train,epochs=100,
          validation_data = (X_valid_scaled, y_valid),
          callbacks=callbacks)


### e.
문제: 알파 드롭아웃으로 모델에 규제를 적용해보세요. 그다음 모델을 다시 훈련하지 않고 MC 드롭아웃으로 더 높은 정확도를 얻을 수 있는지 확인해보세요.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)


model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))

for layer in range(20):
  model.add(keras.layers.Dense(100,
                               activation='selu',
                               kernel_initializer='lecun_normal'))
model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10,activation='softmax'))

optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]


X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds



model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)


MC dropout

In [ ]:
class MCAlphaDropout(keras.layers.AlphaDropout):
  def cal(self,inputs):
    return super().call(inputs, training=True)

AlphaDropout 층 대신 MCAlphaDropout 드롭아웃 층을 사용

In [ ]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    #layer.rate = 드롭아웃 비율
    for layer in model.layers
])

In [ ]:
model.layers[-2].rate

확률예측 함수

In [ ]:
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

확률 기반으로 클래스예측 함수

In [ ]:
def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy